# <img src="https://upload.wikimedia.org/wikipedia/commons/0/0c/Blender_logo_no_text.svg" alt="Blender Logo" width="50">  NeuroBlender: Using Blender for Neuroscience 
This notebook contains the code to import the neuron meshes, the segmentation images and the EM images into Blender. And concisely explains how to scale the images and meshes to the correct size for rendering. This is a starter guide for using Blender to make animation and renders of neurons. I hope it serves!

This code utilizes **CloudVolume** and **MeshParty** to retrieve both images and meshes.  

## Covered Topics: 
- Acquiring the neuron meshes
- Acquiring the segmentation images
- Acquiring the EM images
- Scaling the images to the correct size
- Coloring the neurons appropriately
- Importing the meshes into Blender
- Importing the images into Blender
- Coloring the neurons appropriately

![Objective Scene](tutorial_images/objective.png)

# Prerequisites

Before running this notebook, ensure you have the following dependencies installed:

- **Blender**: Installed (Version used: **4.2.1** on macOS)
- **Python**: Installed (Version used: **3.10.16**)
- **CloudVolume**: Installed (Version used: **11.2.0**)
- **MeshParty**: Installed (Version used: **1.18.2**)
- **Open3D**: Installed (Version used: **0.19.0**)
- **Matplotlib**: Installed (Version used: **3.10.1**)

If you need to install missing dependencies, you can use the following commands if using conda:

```bash
conda create -n blender_env python=3.10
conda activate blender_env
conda install -c conda-forge open3d numpy matplotlib
pip install cloud-volume meshparty caveclient
```

If you prefer using virtualenv:

```bash
python -m venv blender_env
source blender_env/bin/activate
pip install numpy cloud-volume meshparty caveclient open3d matplotlib
```


In [1]:
"""Imports, Run this Block"""
import open3d as o3d
import numpy as np
from cloudvolume import CloudVolume
from meshparty import trimesh_io
from cloudvolume import Bbox
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import os

# Data Acquisition Functions  

This section contains functions to retrieve neuron meshes, segmentation images, and EM images. These functions use specified sources, bounding boxes, and resolutions to fetch and save the required data.  

### Functions:  

- **`get_em_images(em_src, bbox, mip_resolution, save_dir)`**  
  Retrieves EM images from the given source within the specified bounding box and resolution, then saves them to the designated directory.  

- **`get_segmentation_images(seg_src, bbox, mip_resolution, save_dir)`**  
  Acquires segmentation images from the provided source, using the defined bounding box and resolution, and saves them to the target directory.  

- **`acquire_neuron_meshes(seg_ids, color_map, use_bounds, min_bounds, max_bounds, output_dir, offset, trimesh_meta)`**  
  Fetches neuron meshes based on segmentation IDs, applies coloring, and optionally clips them to the specified bounds before saving to the output directory.  


In [2]:
"""Core functions: Run this Block"""
def get_em_images(em_src, bbox, mip_resolution:np.array=[16,16,40], save_dir:str='./objects/default_save/em_images'):
    em_cv = CloudVolume(em_src, mip=mip_resolution, bounded=False, fill_missing=True, use_https=True, progress=True)
    em = em_cv[bbox]

    if save_dir is not None:
        os.makedirs(save_dir, exist_ok=True)
        em.save_images(save_dir,)
    return em

def get_segmentation_images(seg_src, bbox, mip_resolution:np.array=[16,16,40], save_dir:str='./objects/default_save/segmentation_images', seg_ids:np.array=None):
    seg_cv = CloudVolume(seg_src, mip=mip_resolution, bounded=False, fill_missing=True, use_https=True, progress=True)
    seg = seg_cv[bbox]
    if seg_ids is not None:
        # set to 0 all the values that are not in seg_ids
        seg[np.isin(seg, seg_ids, invert=True)] = 0
    if save_dir is not None:
        os.makedirs(save_dir, exist_ok=True)
        seg.save_images(save_dir)
    return seg

def acquire_neuron_meshes(
    seg_ids:np.array, 
    color_map:dict, 
    use_bounds:bool=False, 
    min_bounds:np.array=[0,0,0], 
    max_bounds:np.array=[0,0,0], 
    output_dir:str='./objects/default_save', 
    offset:np.array=[0,0,0], 
    trimesh_meta:trimesh_io.MeshMeta=None):
    """
    Acquires the neuron meshes from the segmentation images and saves them as obj files.
    """
    if trimesh_meta is None:
        raise ValueError("Trimesh_meta must be provided")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for seg_id in seg_ids:
        try:
            print(f"Processing segment ID: {seg_id}")
            mesh = trimesh_meta.mesh(seg_id=int(seg_id))
            
            # Convert trimesh to Open3D format
            vertices = np.array(mesh.vertices)

            faces = np.array(mesh.faces)
            # Downsample the mesh
            #original_num_triangles = len(open3d_mesh.triangles)
            #target_number_of_triangles = max(original_num_triangles // downsampling_factor, 1)
            #simplified_mesh = open3d_mesh.simplify_quadric_decimation(target_number_of_triangles)
            # Filter the vertices based on bounding box
            vertex_flags = []
            filtered_vertices = []
            current_true_idx = 0

            if use_bounds:
                for i, (x, y, z) in enumerate(vertices):
                    if (min_bounds[0] <= x <= max_bounds[0] and
                        min_bounds[1] <= y <= max_bounds[1] and
                        min_bounds[2] <= z <= max_bounds[2]):

                        x -= offset[0]
                        y -= offset[1]
                        z -= offset[2]

                        vertex_flags.append((True, current_true_idx))
                        filtered_vertices.append([x, y, z])
                        current_true_idx += 1
                    else:
                        vertex_flags.append((False, None))

                filtered_faces = []
                for face in faces:
                    remapped_face_indices = []
                    for idx in face:
                        flag, new_idx = vertex_flags[idx]
                        if flag:
                            remapped_face_indices.append(new_idx)
                    
                    if len(remapped_face_indices) == 3: 
                        filtered_faces.append(remapped_face_indices)
            else:
                for x, y, z in vertices:
                    x -= offset[0]
                    y -= offset[1]
                    z -= offset[2]
                    filtered_vertices.append([x, y, z])
                filtered_faces = faces

            open3d_mesh = o3d.geometry.TriangleMesh()
            open3d_mesh.vertices = o3d.utility.Vector3dVector(np.array(filtered_vertices))
            open3d_mesh.triangles = o3d.utility.Vector3iVector(np.array(filtered_faces))

            if seg_id in color_map:
                color = color_map[seg_id]
            else:
                color = [1, 1, 1]  
            colors = np.tile(color, (len(filtered_vertices), 1))

            open3d_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)
            o3d.io.write_triangle_mesh(f'{output_dir}/seg_{seg_id}.obj', open3d_mesh)

            mesh_filename = f'{output_dir}/seg_{seg_id}.obj'
            material_filename = f'{output_dir}/seg_{seg_id}.mtl'
            relative_material_filename = f'seg_{seg_id}.mtl'

            with open(material_filename, 'w') as mtl_file:
                mtl_file.write(f"newmtl SegmentMaterial_{seg_id}\n")
                mtl_file.write(f"Kd {color[0]} {color[1]} {color[2]}\n")  # Diffuse color
                mtl_file.write(f"Ka {color[0]} {color[1]} {color[2]}\n")  # Ambient color
                mtl_file.write(f"Ks 1.0 1.0 1.0\n")  # Specular color
                mtl_file.write("Ns 1000\n")  # Specular exponent
            with open(mesh_filename, 'r') as obj_file:
                obj_lines = obj_file.readlines()
            with open(mesh_filename, 'w') as obj_file:
                obj_file.write(f"mtllib {relative_material_filename}\n") 
                obj_file.write(f"usemtl SegmentMaterial_{seg_id}\n")  
                obj_file.writelines(obj_lines)
            print(f"Processed segment ID: {seg_id} and saved to {mesh_filename} with material {material_filename}")
        except Exception as e:
            print(f"Error processing segment {seg_id}: {e}")
    print("All meshes have been processed, filtered and saved.")

# Acquiring neuron meshes

Blender represents objects as a collection of polygons (triangles essentially). To show a neuron, we need to get the mesh in an obj file, associate a material (color) to it and then import it in Blender.
It is important to track scales between the voxel space and the Blender space. A blender object has to be around the order of magnitude of meters to be correctly displayed.


First, define the segmentation source.

In [3]:
segment_ids = np.array([720575940617386708])
seg_src = 'precomputed://gs://flywire_v141_m783'

Define the colors you want to use for the neurons. Defaults to tab20 color map.

In [ ]:
"""Run this Block"""
num_colors = len(segment_ids)
print(f"Number of unique IDs: {num_colors}")
cmap = plt.get_cmap('tab20', len(segment_ids))  
color_map = {segment_id: cmap(i)[:3] for i, segment_id in enumerate(segment_ids)}  # Map IDs to colors

We instantiate the trimesh mesh meta object. That object will be used to acquire the meshes. We also define the colors we want to use for the neurons.

In [5]:
"""Preparing Trimesh, Run this Block"""

mm = trimesh_io.MeshMeta(
    cv_path=seg_src,
    disk_cache_path="flywire_cache",
    map_gs_to_https=True
)

# Defining the Origin (offset)

Properly defining the neuron offset is **critical** to ensure correct positioning in Blender.  

If the offset is not set, the neuron will be displayed at its voxel coordinates, which are typically on the order of **1e9**. Since Blender does not interpret voxel resolution, it will render the neuron at **1e9 meters**, causing extreme misplacement. Getting to see the neuron is feasible but will take a lot of time if you have not set the offset correctly.

To prevent this, we define the offset **in voxel space** and convert it to nm, ensuring accurate positioning and scale within Blender.  


In [6]:
offset = np.array([146739, 42915, 900]) # TODO: change freely. This will mean this point will be at the origin in Blender space.
em_resolution = np.array([4,4,40]) # TODO: change accordingly to the resolution of the EM images.
blender_origin = offset * em_resolution # to convert to nm

# Mesh Acquisition.

They will be saved each as their own obj file, with a matching mtl file. You can define a bounding box to limit the mesh to a specific region.

In [ ]:
acquire_neuron_meshes(segment_ids, color_map, use_bounds=False, output_dir='./default_save/neuron_meshes/', offset=blender_origin, trimesh_meta=mm)

# Importing `.obj` Files in Blender

## Steps for Importing:
1. **Import as `.obj`**:  
   - Go to **Files > Import > Wavefront (.obj)**.  
   - **Select all neurons wanted**. 

2. **Import Axis Configuration**:  
   - **Forward Axis: `Y`**  
   - **Up Axis: `Z`**  

3. **After Importing**:  
   - The **neuron should be huge**.  
   - Add an **Empty** object.  
   - Make the neuron a **child** of the empty:
     - Press `Shift + A`, select **Empty**.  
     - Select both neuron and empty, then press `Cmd + P`, choose **Default Parenting**.  

4. **Scaling**:  
   - Scale the empty by **1e-4 or 1e-3**.  
   - **Recommended: scale by`1e-4`**. Meaning `1 meter` in Blender = `10000 nm` in real life.  

5. **Visibility Check**:  
   - If the object is not visible:  
     1. Adjust **scales** and check **Dimensions tab**.  
     2. Verify the **viewport visualization mode**.  

## **Performance Tip**:
- To **reduce load**, scale at **import time**.  
- Keep objects in **nm units** and use the empty for Blender scaling.  
- You can also adjust **decimation** in `acquire_neuron_meshes()`.  



![Example Neurons Imported](tutorial_images/example_neurons.png)

# Acquiring EM and Segmentation Images  

We use [CloudVolume](https://github.com/seung-lab/cloud-volume) to obtain EM and segmentation images. First, we define the source of the images and the bounding box for the region of interest.  


In [9]:
em_src = 'precomputed://https://bossdb-open-data.s3.amazonaws.com/flywire/fafbv14'
seg_src = 'precomputed://gs://flywire_v141_m783'

Define here the image resolution you want (the quality of the image). 

Tip: Higher quality images will take longer to load and render in Blender. If you build a stack of images, consider only using the first image in high resolution and the rest in low resolution.

In [10]:
seg_image_resolution = np.array([16,16,40]) # this affects the resolution of the segmentation images. Doesn't affect the size in Blender, just quality.
em_image_resolution = np.array([16,16,40]) # this affects the resolution of the EM images. Same as above.

# Lower resolution would be [32,32,40] for example. You can check with CloudVolume how many mip levels are available.

Here we define the box we want to have of images. In this example, we want a cube of images and we want it to be centered on the origin and the images going down the Z-axis.

In [11]:
first_image_ctr = offset
em_resolution = np.array([4,4,40]) # Voxel resolution of the EM images
img_size_x = 1000 
img_size_y = 1000
img_size_z = 100   

In [12]:
bbox = Bbox([first_image_ctr[0] - img_size_x, first_image_ctr[1] - img_size_y, first_image_ctr[2] - 2*img_size_z+1], 
            [first_image_ctr[0] + img_size_x, first_image_ctr[1] + img_size_y, first_image_ctr[2] + 1]) 
bbox *= em_resolution
bbox.unit = 'nm'
size_x_nm = 2*img_size_x*em_resolution[0]
size_y_nm = 2*img_size_y*em_resolution[1]
size_z_nm = 2*img_size_z*em_resolution[2]

In [ ]:
seg_cutout = get_segmentation_images(seg_src, bbox, mip_resolution=seg_image_resolution, save_dir='./default_save/segmentation_images/', seg_ids=segment_ids)
print(f"Size of the segmentation image in nm: {size_x_nm} x {size_y_nm} x {size_z_nm}")

In [ ]:
em_cutout = get_em_images(em_src, bbox, mip_resolution=em_image_resolution, save_dir='./default_save/em_images/')
print(f"Size of the EM image in nm: {size_x_nm} x {size_y_nm} x {size_z_nm}. Images are spaced by {em_resolution[2]} nm in the Z-axis and inverted in the Y-axis (compared to neuroglancer).")
print(f"Import image mesh plane (Shift + A, then Image, then Image Mesh Plane). Set height to {size_x_nm} m and Z-offset to {em_resolution[2]} m.")

When importing the images, use **<span style="color:red;">offset_direction = -Z</span>** and the distance between images to be **<span style="color:red;">40 m</span>** and align to the **<span style="color:red;">+Z axis</span>**.  
Also use the height corresponding to **<span style="color:red;">size_x_nm</span>**.  

Then add an **<span style="color:red;">empty</span>** for scaling, scale it with the same ratio you used for the neurons (**<span style="color:red;">1e-4</span>**).  
**<span style="color:red;">Invert the Y-axis</span>** of the empty to be exactly as in **<span style="color:red;">Neuroglancer</span>**.  


# Theoretical end result.

You should end up with something like this.
You should expect to have the section 000 to be the lowest in the Z-direction as CloudVolume saves them in the order of the Z-axis.
I recommend using a collection for each of the objects and having a different scaling empty for the EM and the neuron meshes.

![End Result](tutorial_images/end_result.png)


# Going further in cool visuals

The neurons as imported look jagged. You can apply smoothing to them by right clicking on the mesh and selecting "Shade Smooth".

One cool thing to do is to melt the EM stack. To hack this, you can just create a cube (Shift + A, then Mesh, then Cube) and scale it to the size of the EM stack.
Then make it transparent and disable it in Shadows and View (Select the cube, then in Object Properties, disable Shadow). The cube should now be invisible. In the below photo the Ray visibility parameters, disable both camera and Shadow.




![Cube Properties](tutorial_images/cube_in_blender.png)



You can then add a Boolean modifier to all the images in respect to the cube. (Select the image, then in the Modifier tab, add a Boolean modifier, then select the cube as the target, and set the operation to Difference.)


![Boolean Modifier](tutorial_images/modifier.png)

If you have a collection, you can also add a Boolean modifier to the collection in respect to the cube, instead of applying it to each image which get tedious.



Go in Scripting tab (top middle-ish), copy-paste and run the following script.


In [ ]:
import bpy

cube = bpy.data.objects.get("Cube") # TODO:CHANGE THIS TO THE NAME OF YOUR INTERSECTING OBJECT

# Get the "EM_images" collection
collection = bpy.data.collections.get("EM_images") # TODO: CHANGE THIS TO THE NAME OF YOUR COLLECTION

if cube and collection:
    for obj in collection.objects:  
        if obj.type == 'MESH':
            # Check if a Boolean modifier already exists, otherwise add one
            boolean_mod = None
            for mod in obj.modifiers:
                if mod.type == 'BOOLEAN':
                    boolean_mod = mod
                    break
            
            if not boolean_mod:
                boolean_mod = obj.modifiers.new(name="Boolean_Cut", type='BOOLEAN')

            boolean_mod.object = cube 
            boolean_mod.operation = 'DIFFERENCE' 
            print(f"Boolean modifier applied to {obj.name}")
else:
    print("Error: Cube or EM_images collection not found! Check object names.")

# Last but not least: Keyframes.

Right now we have cool objects, we didn't even have to build them, just import them.
But they are static... That does not make a great movie.

The core of video animation in Blender is setting starting points or ending points for a transition of an object: using keyframes.

To make an object move, you set a keyframe on the first frame where the object should start the movement, setting the keyframe for the object's position, and then another keyframe on the last frame where the object should end the movement, setting the keyframe for the object's position again. There is a full tutorial on the Seung Lab slack, ask me for access.

<!-- <video controls width="1200">
  <source src="tutorial_images/example_keyframes.mov" type="video/mp4">
  Your browser does not support the video tag.
</video> -->



This is the the most basic way of rendering neurons and EM data. Rendering more neurons, with larger sections, is a matter of diminshing memory usage, the complexity of each frame rendering.
I will add the code to allow cluster rendering of a project in a future update.

Thanks for reading! Contact me at levisseraphael0@gmail.com for any questions or suggestions. Thanks to the Seung Lab for the data, Sven Dorkenwald and the CaveClient team for meshparty, William Silversmith for his efficient cloudvolume library. And of course, Pr. Sebastian Seung for motivating this project.